In [ ]:
!pip install statannot
!pip install spectral

import io
import os
import scipy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn import datasets
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import cohen_kappa_score
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from google.colab import files
import seaborn as sns
from sklearn.neural_network._multilayer_perceptron import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from osgeo import gdal
from skimage import exposure
from sklearn import metrics
from matplotlib import colors
from spectral import*


     |████████████████████████████████| 215kB 3.4MB/s 


In [ ]:
# Define functions
# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

#-------------------------------------------------------------------------------
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)

    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type

    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)


#-------------------------------------------------------------------------------
# Plot confusion matrix for classification result
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Greys):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    #if not title:
    #    if normalize:
    #        title = 'Normalized confusion matrix'
    #    else:
    #        title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize=(5.5, 4.9), frameon=True)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.ax.tick_params(labelsize=13)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=0, ha="center",
             rotation_mode="anchor", fontsize=12)
    
    plt.setp(ax.set_xlabel('Predicted label', fontsize=18))

    plt.setp(ax.get_yticklabels(), rotation=90, ha="center",
             rotation_mode="anchor", fontsize=12)
    
    plt.setp(ax.set_ylabel('True label', fontsize=18))
    
    plt.setp(ax.set_title(title, fontsize=18))

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt), size=13,
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

#-------------------------------------------------------------------------------
# Export classification results to GeoTIFF
from osgeo import gdal,osr
import numpy as np

def save_raster(path, img, prj, geotran, format='GTiff', dtype = gdal.GDT_Float32):
    rows, cols = img.shape
    # Initialize driver & create file
    driver = gdal.GetDriverByName(format)
    dataset_out = driver.Create(path, cols, rows, 1, dtype)
    dataset_out.SetGeoTransform(geotran)
    dataset_out.SetProjection(prj)
    # Write file to disk
    dataset_out.GetRasterBand(1).WriteArray(img.real)
    dataset_out.FlushCache()
    dataset_out=None

#-------------------------------------------------------------------------------
# Define the derivative calculation function
def derivative(f, x, method='central', interval=1):
    '''Compute the difference formula for f'(x) with step size interval.
    Parameters:
    -----------
    f: Function
       Vectorized function of one variable
       
    x: Compute derivative at wavelength x
       method: string - 'forward', 'backward' or 'central'
    
    interval: Step size in difference formula
    '''
    
    if method == 'central':
        return (f(x + interval) - f(x - interval))/(2*interval)
    elif method == 'forward':
        return (f(x + interval) - f(x))/interval
    elif method == 'backward':
        return (f(x) - f(x - interval))/interval
    else:
        raise ValueError("Method must be 'central', 'forward', or 'backward'.")



In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Read reflectance data for derivative calculationplt.style.use('seaborn-dark')
import time
from google.colab import drive
drive.mount('/content/gdrive')
#plt.style.use('classic')
plt.style.use('seaborn-ticks')

start_time = time.process_time()

# Preprocess the data
ref_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/2019_Stress_T0_Deriv_24.csv')
ref_data_ori = ref_data
status = encode_text_index(ref_data, "Status")
x_ml = ref_data.drop(['Sample', 'Status'], axis=1)
y_ml = ref_data['Status']
wavelength = np.arange(440,802,2)

# Scale the data and split the training and test samples
x_ml_tran = x_ml.T
x_ml_tran_scaled = MinMaxScaler().fit_transform(x_ml_tran)
x_ml_scaled = x_ml_tran_scaled.T
x_ml_train, x_ml_test, y_ml_train, y_ml_test = train_test_split(x_ml_scaled, y_ml, test_size = 0.25, random_state = 42, stratify = y_ml)

# Plot the spectral curves
main_font = 22
Sub_font = 18

# Calculate the mean spectral curves
ref_mean = ref_data.groupby("Sample").mean()
wavelength = np.arange(440,800,2)

# Plot the mean spectra
plt_ref_mean = (ref_mean.iloc[0:10, 1:181].values).transpose()
plt_ref_mean = pd.DataFrame(plt_ref_mean, index=wavelength)
plt_ref_mean.plot(figsize=(8, 7), color=sns.set_palette("husl", 10))
plt.title("Mean Spectral Derivative", fontsize=main_font)
plt.xlabel('Wavelength (nm)', fontsize=main_font)
plt.ylabel('Spectral Derivative', fontsize=main_font)
plt.grid(color='grey', linestyle='dotted', linewidth=0.55)
plt.tick_params(direction = 'in', color='black')
plt.xticks(fontsize=Sub_font)
plt.xlim(430, 810)
plt.ylim(0, 0.4)
plt.yticks(fontsize=Sub_font, rotation=90)
plt.legend(loc='upper left', frameon=False, fontsize = Sub_font,
           labels = ['Day 00', 'Day 03','Day 06',
                     'Day 09', 'Day 12', 'Day 15',
                     'Day 18', 'Day 21', 'Day 24',
                     ],
           ncol=2)
plt.savefig('/content/gdrive/My Drive/Mean_Spectral.png', dpi=300)
end_time = time.process_time()
elapsed_time = (end_time - start_time)
print("Processing time:", round(elapsed_time,3), "Secs")

In [ ]:
# Multilayer perceptron deep (MLP) neural network classification for full images
#from skopt import BayesSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
from google.colab import drive
from sklearn.metrics import accuracy_score
import pickle
plt.style.use('seaborn-dark')

start_time = time.time()

# Setup parameters for MLP model tuning
mlp_parameters = {
    'hidden_layer_sizes': [(300,300), (300,50), (200,200), (100,100), (50,50)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.00001, 0.0001, 0.001],
    'learning_rate': ['constant','adaptive', 'invscaling'],
    #'batch_size': [200, 300, 400, 500, 600, 700],
    #'n_iter_no_change': [30]
}

mlp_grid = RandomizedSearchCV(estimator = MLPClassifier(max_iter=1000),
                              param_distributions=mlp_parameters, n_iter=20, cv = 10, verbose = 1, n_jobs=-1)

mlp_grid.fit(x_ml_train, y_ml_train)

# Print the tuning result and get the best parameters
print("The best parameters are %s with a score of %0.3f"
      % (mlp_grid.best_params_, mlp_grid.best_score_))

print("The best MLP model:")
print(mlp_grid.best_estimator_)

# Print the trainng results
mlp_means = mlp_grid.cv_results_['mean_test_score']
mlp_stds = mlp_grid.cv_results_['std_test_score']
for mean, std, params in zip(mlp_means, mlp_stds, mlp_grid.cv_results_['params']):
    print("(+%0.3f/-%0.03f) for %r"% (mean, std * 2, params))

#Plot the training result graph
plt.plot(mlp_grid.cv_results_['mean_test_score'])
plt.title('Model train vs. Validation loss')
plt.xlabel('epochs')
plt.ylabel('Mean square error')
plt.legend(['Train MSE', 'validation MSE'], loc='upper left')
plt.show()

# Save the trained MLP model as a pickle file
drive.mount('/content/gdrive')
path = F"/content/gdrive/My Drive/mlp_spectralderiv_T0_24.pkl" 
joblib.dump(mlp_grid, open(path, 'wb'))

# Calculate training time
end_time = time.time()
elapsed_time = (end_time - start_time)
print("Processing time:", round(elapsed_time,3), "Secs")


#-------------------------------------------------------------------------
# Load the model and perform MLP classification
mlp_classifier =  joblib.load(open(path, 'rb'))

y_ml_pred = mlp_classifier.predict(x_ml_test)

kappa_mlp = cohen_kappa_score(y_ml_test, y_ml_pred)
Overall_acc = accuracy_score(y_ml_test, y_ml_pred)
print('Overall Accuracy:', round(Overall_acc*100, 3))
print('kapa Cofficient:', round(kappa_mlp,3))

# Plot confusion matrix
np.set_printoptions(precision=2)

#Plot normalized confusion matrix
plot_confusion_matrix(y_ml_test, y_ml_pred, classes=status, normalize=True,
                      title='Normalized matrix - DNN-Deriv')

plt.savefig("/content/gdrive/My Drive/mlp_spectralDeriv_matrix_T0_24.png", dpi=300)
plt.show()

# Save runtime and accuracy to CSV
with open('/content/gdrive/My Drive/mlp_spectralderiv_accuracy_' + 'T0' + '_24.txt', 'a', newline='') as txtfile:
  #row_write = csv.writer(csvfile, delimiter=' ')
  txtfile.write('Runtime: ' + str(round(elapsed_time, 0)) + '\n')
  txtfile.write('Overall Accuracy: ' + str(round(Overall_acc*100, 3)) + '\n')
  txtfile.write('Kappa Coefficient: ' + str(round(kappa_mlp, 3)))

In [ ]:
# Read origial raster dataset and stack image bands
from google.colab import drive
treat_day = 'D-24'
drive.mount('/content/gdrive')
img_ori_name = "/content/gdrive/My Drive/T01_20190718_Ref_Sm_Res_Sub_440-800_segmentation.tif"

raster_dataset = gdal.Open(img_ori_name, gdal.GA_ReadOnly)
geo_transform = raster_dataset.GetGeoTransform()
proj = raster_dataset.GetProjectionRef()
n_bands = raster_dataset.RasterCount

hsi_img = np.moveaxis(raster_dataset.ReadAsArray()[:251], 0, -1)

#hsi_img = exposure.rescale_intensity(bands_data, out_range=(0,1))
plt.style.use('seaborn-ticks')

# Display RGB image
# Produce mask
vmin = 0.15
vmax = 0.4
rgb_img = np.dstack([hsi_img[:, :, 180], hsi_img[:, :, 120], hsi_img[:, :, 55]])
#rgb_img = np.dstack([hsi_img[:, :, 120], hsi_img[:, :, 55], hsi_img[:, :, 5]])

plt.figure(figsize=(6.5, 6))
rgb_img = exposure.rescale_intensity(rgb_img, out_range=(0,1))
rgb_img[(hsi_img[:, :, 180] <= vmin)] = 1e+20
plt.title(treat_day, fontsize=18)
plt.imshow(rgb_img, alpha=1)
plt.xticks(ticks=[], labels=None)
plt.yticks(ticks=[], labels=None)
plt.grid(b=None)
savefile = '/content/gdrive/My Drive/T0_rgb_img_' + treat_day
plt.savefig(savefile + '.png', dpi=150)
plt.show()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Perform image classification
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
from google.colab import drive
from sklearn import svm
from pylab import *
import pickle
import gdal
import os

# Load the pickled model
plt.style.use('seaborn-ticks')
drive.mount('/content/gdrive')

day_list = ['D-00', 'D-03', 'D-06', 'D-09', 'D-12', 'D-15', 'D-18', 'D-21', 'D-24']
date_list = ['20190617', '20190620', '20190623', '20190626', '20190630', '20190704', '20190708', '20190712', '20190718']
mask_list = [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.2, 0.15]
for treat_day, treat_date, mask_val in zip(day_list, date_list, mask_list):
  img_ori_name = '/content/gdrive/My Drive/T01_' + treat_date + '_Ref_Sm_Res_Sub_440-800_segmentation.tif'

  raster_dataset = gdal.Open(img_ori_name, gdal.GA_ReadOnly)
  geo_transform = raster_dataset.GetGeoTransform()
  proj = raster_dataset.GetProjectionRef()
  n_bands = raster_dataset.RasterCount
  hsi_img = np.moveaxis(raster_dataset.ReadAsArray()[:251], 0, -1)

  path = F"/content/gdrive/My Drive/mlp_fullspectra_T0_24.pkl"
  mlp_classifier =  joblib.load(open(path, 'rb'))

  n_row, n_col, n_band = hsi_img.shape
  classified_img = np.empty(shape=(n_row, n_col), dtype='uint8')
  for i in np.arange(n_row):
    for j in np.arange(n_col):
      x_ml_tran = pd.DataFrame(hsi_img[i,j,:])
      x_scaled = x_ml_tran.T
      classified_img[i,j] = mlp_classifier.predict(scaler.transform(x_scaled))

  # Plot the classified image
  import os
  import gdal
  from pylab import *
  plt.style.use('seaborn-ticks')

  classified_img_mask = classified_img
  classified_img_mask = np.ma.masked_array(classified_img_mask, mask=(hsi_img[:, :, 180] <= mask_val))

  # Plot classified image
  color_map = plt.cm.get_cmap('RdYlGn', 9)
  cmap = color_map.reversed()
  class_name = ['D-00', 'D-03', 'D-06', 'D-09', 'D-12', 'D-15', 'D-18', 'D-21', 'D-24']
  plt.figure(figsize=(6.5, 6))
  img1 = plt.imshow(classified_img_mask, interpolation='none', cmap=cmap) #plt.cm.Greens, plt.cm.YlGn, plt.cm.get_cmap('Greens', 9)
  plt.title(treat_day, fontsize=18)
  cbar = plt.colorbar(img1)
  cbar.set_label(label="Day of Treatment", size=18)
  cbar.ax.tick_params(labelsize=16, rotation=0)
  cbar.ax.set_yticklabels(labels=class_name, va="center")
  plt.xticks(ticks=[], labels=None)
  plt.yticks(ticks=[], labels=None)
  plt.grid(b=None)
  savefile = '/content/gdrive/My Drive/T0_img_fullspectra_mlp_' + treat_day 
  plt.savefig(savefile + '.png', dpi=150)
  plt.show()

  # Export classification results to GeoTIFF
  save_tiff_file = savefile + '.tif'
  prj = raster_dataset.GetProjection()
  geotran = raster_dataset.GetGeoTransform()
  save_raster(path=save_tiff_file, img = classified_img_mask, prj=prj, geotran=geotran, format='GTiff', dtype=gdal.GDT_Byte)